# **Clase 1: Inteligencia Artificial para Investigación de Mercados Internacionales: Ética, Sesgos y Prompts Efectivos**



## **Autores**

Gerencia de Inteligencia Comercial - Coordinación Analítica:

1. Nestor Enrique Forero Herrera: nforero@procolombia.co
2. María Paula Díaz Nuñez: mdiaz@procolombia.co
3. Nicolás Rivera Garzón: nrivera@procolombia.co

# **Tabla de Contenido**

## Configuración Inicial
1. [Configurar Google Drive](#configurar-google-drive)
2. [Instalación de librerías](#instalación-de-librerías)
3. [Importar las llaves de las APIs de Google y OpenAI](#importar-las-llaves-de-las-apis-de-google-y-openai)

## Funciones para Prompts
4. [Funciones para realizar prompts con OpenAI - ChatGPT](#funciones-para-realizar-prompts-con-openai---chatgpt)
5. [Funciones para realizar prompts con Google - Gemini](#funciones-para-realizar-prompts-con-google---gemini)

## Prompt Engineering
6. [Utiliza delimitadores para indicar claramente distintas partes del prompt](#utiliza-delimitadores-para-indicar-claramente-distintas-partes-del-prompt)
7. [Define la longitud o formato de la respuesta esperada](#define-la-longitud-o-formato-de-la-respuesta-esperada)
8. [Proporciona texto de referencia](#proporciona-texto-de-referencia)
9. [Divide la tarea en pasos individuales](#divide-la-tarea-en-pasos-individuales)

## Caso de estudio: Exportación de productos de café a China
10. [Traducción y resumen de videos en otro idioma](#traducción-y-resumen-de-videos-en-otro-idioma)
11. [Juguemos con la temperatura](#juguemos-con-la-temperatura)
12. [Traducción de archivos Excel](#traducción-de-archivos-excel)

## **1. Configurar Google Drive** <a name="configurar-google-drive"></a>

Pasos detallados:

1. Ejecuta la siguiente celda en tu notebook de Google Colab.
2. Se te pedirá que hagas clic en un enlace para autorizar a Google Colab a acceder a tu Google Drive.
3. Después de hacer clic en el enlace, selecciona tu cuenta de Google y permite el acceso.
4. Copia el código de autorización que se te proporciona.
5. Pega el código de autorización en el cuadro de entrada en tu notebook de Google Colab y presiona Enter.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Elegir directorio
%cd "/content/drive/MyDrive/AI_para_Empresas_Exportadoras/Clase_1/Insumos"

/content/drive/MyDrive/AI_para_Empresas_Exportadoras/Clase_1/Insumos


## **2. Instalación de librerías** <a name="instalación-de-librerías"></a>

In [5]:
# Instalar todas las librerías necesarias
!pip install openai google-generativeai ipython tiktoken requests pandas numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [6]:
# Importar librerías
# OpenAI
from openai import OpenAI

# Google
import google.generativeai as genai

# Respuestas en formato claro
from IPython.display import display, Markdown

# Conteo de Tokens OpenAI
import tiktoken

# Web
import requests

# Básicos
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("Todas las librerías se han instalado e importado correctamente.")

Todas las librerías se han instalado e importado correctamente.


## **3. Importar las llaves de las APIs de Google y OpenAI** <a name="importar-las-llaves-de-las-apis-de-google-y-openai"></a>


In [8]:
from google.colab import userdata
gemini_key = userdata.get('gemini_api_key')

In [9]:
chatgpt_key = userdata.get('openai_api_key')

## **4. Funciones para realizar prompts con Open AI - ChatGPT** <a name="funciones-para-realizar-prompts-con-openai---chatgpt"></a>

In [10]:
# Función para realizar conexión a la API de OpenAI
def create_openai_client(api_key):
    """
    Crea el cliente de OpenAI y verifica si la conexión es exitosa.

    Parámetros:
    - api_key (str): La clave de la API de OpenAI.

    Retorna:
    - client (OpenAI): El objeto cliente de OpenAI, o None en caso de error.
    """
    try:
        # Crear el objeto cliente de OpenAI utilizando la clave de la API proporcionada.
        client = OpenAI(api_key=api_key)

        # Realizar una solicitud de prueba para verificar la conexión a la API listando los modelos disponibles.
        client.models.list()
        print("Conexión a OpenAI exitosa.")

        # Obtener la lista de modelos disponibles a través de la API y mostrarlos.
        models = client.models.list()
        print("Modelos disponibles:")
        for model in models:
            print(f"- {model.id}")

    # Manejo de excepciones en caso de error en la conexión o solicitud.
    except Exception as e:
        print("Error al conectar con OpenAI:", str(e))
        client = None

    # Devolver el objeto cliente de OpenAI o None si hubo un error.
    return client

In [11]:
# Función para ejecutar prompts en OpenAI
def get_completion_open_ai(prompt, model="gpt-4o-mini", temperature=0.1, max_tokens=None, n=1, stop=None):
    """
    Función para obtener una respuesta del modelo GPT de OpenAI dado un prompt.

    Parámetros:
    - prompt (str): El texto de entrada al que el modelo debe responder.
    - model (str): El nombre del modelo a utilizar. Por defecto, "gpt-4o-mini".
    - temperature (float): Grado de aleatoriedad en la respuesta del modelo (0-1). Por defecto, 0.1.
                         Ajusta la creatividad de la salida del modelo. Un valor más bajo (cercano a 0) hará que el modelo sea más determinista y conservador,
                         generando respuestas más predecibles y repetitivas. Un valor más alto (cercano a 1) permitirá al modelo tomar más riesgos,
                         produciendo respuestas más variadas y creativas.
    - max_tokens (int): Número máximo de tokens en la respuesta. Por defecto, None (sin límite).
    - n (int): Número de respuestas a generar. Por defecto, 1.
    - stop (list of str): Secuencia o lista de secuencias en las que el modelo debe detenerse. Por defecto, None.

    Retorna:
    - str o list of str: La(s) respuesta(s) generada(s) por el modelo.
    """

    # Verificar si el cliente fue creado exitosamente
    if client is None:
        print("No se puede obtener la respuesta debido a un error en la conexión con OpenAI.")
        return None

    # Construir el mensaje para el modelo en el formato requerido
    messages = [{"role": "user", "content": prompt}]

    try:
        # Llamar a la API de OpenAI para obtener la respuesta
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            n=n,
            stop=stop
        )

        # Extraer las respuestas generadas
        if n == 1:
            # Si se genera una sola respuesta, devolverla como una cadena
            return response.choices[0].message.content.strip()
        else:
            # Si se generan múltiples respuestas, devolverlas como una lista de cadenas
            return [choice.message.content.strip() for choice in response.choices]

    except Exception as e:
        print("Error al obtener la respuesta del modelo:", str(e))
        return None

## **5. Funciones para realizar prompts con Google - Gemini** <a name="funciones-para-realizar-prompts-con-google---gemini"></a>

In [12]:
# Función para realizar conexión a la API de Google
def create_gemini_client(api_key):
    """
    Configura el cliente de Gemini con la clave de API proporcionada.

    Parámetros:
    - api_key (str): La clave de la API de Gemini.

    Retorna:
    - None
    """
    try:
        # Establecer la clave de API en las variables de entorno
        os.environ["GEMINI_API_KEY"] = api_key

        # Configurar la biblioteca genai con la clave de API proporcionada
        genai.configure(api_key=api_key)

        # Intentar una solicitud de prueba para verificar la conexión a la API de Gemini
        model = genai.GenerativeModel("gemini-1.0-pro-latest")
        model.generate_content("Test connection")

        print("Conexión a Gemini exitosa.")

        # Listar y mostrar los modelos disponibles que soportan generación de contenido
        print("Modelos disponibles:")
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                print(m.name)

    # Manejo de excepciones en caso de error en la conexión o solicitud
    except Exception as e:
        print("Error al conectar con Gemini:", str(e))

In [13]:
# Función para ejecutar prompts en Gemini
def get_gemini_completion(prompt, model_name="gemini-1.5-flash", temperature=0.5, max_tokens=None, stop_sequences=None, candidate_count=1):
    """
    Función para obtener una respuesta del modelo de Gemini dado un prompt.

    Parámetros:
    - prompt (str): El texto de entrada al que el modelo debe responder.
    - model_name (str): El nombre del modelo a utilizar. Por defecto, "gemini-1.5-flash".
    - temperature (float): Grado de aleatoriedad en la respuesta del modelo (0-1). Por defecto, 0.1.
                         Ajusta la creatividad de la salida del modelo. Un valor más bajo (cercano a 0) hará que el modelo sea más determinista y conservador,
                         generando respuestas más predecibles y repetitivas. Un valor más alto (cercano a 1) permitirá al modelo tomar más riesgos,
                         produciendo respuestas más variadas y creativas.
    - max_tokens (int): Número máximo de tokens en la respuesta. Por defecto, None (sin límite).
    - stop_sequences (list): Secuencia o lista de secuencias en las que el modelo debe detenerse. Por defecto, None.
    - candidate_count (int): Número de candidatos a generar. Por defecto, 1.

    Retorna:
    - str: La respuesta generada por el modelo.
    """
    try:
        # Crear el modelo con el nombre especificado
        model = genai.GenerativeModel(model_name)

        # Configurar la generación de contenido
        generation_config = genai.types.GenerationConfig(
            candidate_count=candidate_count,
            stop_sequences=stop_sequences,
            max_output_tokens=max_tokens,
            temperature=temperature
        )

        # Generar contenido basado en el prompt utilizando la configuración especificada
        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )

        # Devolver el texto de la respuesta generada
        return response.text
    except Exception as e:
        # Manejo de excepciones en caso de error al obtener la respuesta del modelo
        print("Error al obtener la respuesta del modelo:", str(e))
        return None


**Crear conexión con OpenAI**

In [14]:
# Crear un cliente para la API de OpenAI utilizando la clave de API proporcionada
client = create_openai_client(chatgpt_key)

Conexión a OpenAI exitosa.
Modelos disponibles:
- dall-e-3
- gpt-4o-2024-05-13
- gpt-4-turbo-2024-04-09
- gpt-4-1106-preview
- dall-e-2
- gpt-4o
- gpt-4-turbo
- tts-1-hd-1106
- tts-1-hd
- gpt-4-0125-preview
- babbage-002
- gpt-4-turbo-preview
- text-embedding-3-small
- text-embedding-3-large
- gpt-3.5-turbo
- gpt-3.5-turbo-instruct
- gpt-3.5-turbo-instruct-0914
- whisper-1
- text-embedding-ada-002
- gpt-3.5-turbo-16k
- davinci-002
- tts-1-1106
- gpt-3.5-turbo-0125
- tts-1
- gpt-3.5-turbo-1106
- gpt-4o-mini-2024-07-18
- gpt-4o-mini
- gpt-4-0613
- gpt-4


In [15]:
# Definición del prompt de prueba
prompt_saludo = """
Hola,

Estamos en un Workshop para el fortalecimiento del uso de la inteligencia artificial dirigido por el Ministerio de Comercio, Industria y Turismo a través de ProColombia.

En esta sesión vamos a realizar una investigación de mercado para exportar frutas a China.

Saluda a tus estudiantes
"""

# Obtener una respuesta del modelo de OpenAI utilizando el prompt definido
respuesta = get_completion_open_ai(prompt_saludo)

# Mostrar el contenido de la respuesta como Markdown
display(Markdown(respuesta))

¡Hola a todos!

Es un placer estar aquí con ustedes en este Workshop para el fortalecimiento del uso de la inteligencia artificial. Estoy emocionado de que estemos trabajando juntos en la investigación de mercado para exportar frutas a China. Este es un tema muy relevante y lleno de oportunidades.

Espero que todos estén listos para aprender y compartir ideas. ¡Vamos a aprovechar al máximo esta sesión!

Saludos a todos.

**Crear conexión con Google**

In [16]:
# Crear un cliente para la API de Gemini utilizando la clave de API proporcionada
create_gemini_client(gemini_key)

Conexión a Gemini exitosa.
Modelos disponibles:
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash


In [17]:
# Obtener una respuesta del modelo de Gemini utilizando el prompt definido
respuesta = get_gemini_completion(prompt=prompt_saludo)

# Mostrar el contenido de la respuesta como Markdown
display(Markdown(respuesta))

¡Buenos días a todos! 

Es un placer darles la bienvenida a este taller sobre el fortalecimiento del uso de la inteligencia artificial, organizado por el Ministerio de Comercio, Industria y Turismo a través de ProColombia. 

Hoy nos embarcaremos en una investigación de mercado para explorar las posibilidades de exportar frutas a China. 

Me emociona mucho trabajar con ustedes en este proyecto y aprender juntos cómo la inteligencia artificial puede ser una herramienta poderosa para impulsar el éxito de nuestras exportaciones. 

¡Comencemos! 


**Función de limpieza de prompts**

In [18]:
def clean_text(text):
    """
    Limpia el texto eliminando líneas en blanco y espacios adicionales.

    Parámetros:
    text (str): El texto a limpiar.

    Retorna:
    str: El texto limpiado sin líneas en blanco ni espacios adicionales.
    """
    # Dividir el texto en líneas usando el salto de línea como delimitador
    lines = text.split('\n')

    # Eliminar líneas que están vacías después de quitar espacios en blanco al principio y al final
    # Esto también elimina cualquier línea que contenga solo espacios en blanco
    cleaned_lines = [line.strip() for line in lines if line.strip()]

    # Unir las líneas limpiadas en un solo bloque de texto
    # Se usa '\n' para agregar un solo salto de línea entre las líneas
    cleaned_text = '\n'.join(cleaned_lines)

    return cleaned_text

# **Prompt Engineering** <a name="prompt-engineering"></a>

## **6. Utiliza delimitadores para indicar claramente distintas partes del prompt** <a name="utiliza-delimitadores-para-indicar-claramente-distintas-partes-del-prompt"></a>

In [19]:
prompt_delimitadores = """

Resume y traduce a español el texto en inglés delimitado por {} en 5 bullets. La fuente del texto es Maersk y su tema es acerca de documentos necesrios para la exportación de bienes a Estados Unidos.


{Customs documents to ship to or from the US - everything you need to know
27 September 2023
Maersk Customs Services North America
Share
A smiling man standing with a writing board against the background of a warehouse with cardboard boxes.
Want to ship to the US but don’t know which documents you need to get through customs clearance? Don’t worry, you’re not alone! Many shippers face the same problem. In this article, we explain the key documents you need to ship smoothly to and from the United States.

Being prepared with strong customs processes gives you peace of mind, but more importantly, it ensures you don't encounter delays, detention, or additional charges and penalties against your cargo.

Which export customs documents do you need for the United States?
These are the documents you'll need to be aware of when exporting goods out of the United States:

1. Pro forma invoice or commercial invoice
You will need either a pro forma invoice or a commercial invoice if you export goods from the US.

The pro forma invoice is a document sent to buyers in advance of any shipment or delivery of cargo. Because it is an early-stage document, it is often used as a negotiation tool between buyer and seller – giving estimate quotes for the shipment of goods. The proforma invoice is generally used if there is no sale of an item between the shipper and importer. In other words, a value for customs purposes only.

The pro forma invoice will eventually be used to create the commercial invoice, as it contains similar information as a commercial invoice. You do not need both a pro forma invoice and a commercial invoice. One is sufficient.

The commercial invoice is an anchor document throughout foreign trade. It contains detailed information about the goods you're shipping, their manufacturer, origin, destination, and HTS code.

The primary purpose of a commercial invoice is to advise customs on the commodity you are importing along with the cost for statistical purposes. The value of the goods is used to calculate duties and must be within a specific parameter for statistical purposes, or it could cause a census warning. It's important to complete the commercial invoice in English, as US customs requires.

Details you can expect in a pro forma and commercial invoice
Details of buyer and seller
Detailed description of goods (itemized in English)
Any appropriate Harmonised Tariff Schedule (HTS) code
Itemized purchase price cost of goods in currency of purchase
Terms of payment
Details of delivery
Currency used for quote
Quantities and weights
Country of origin
Any assists, rebates, drawbacks, etc.
Some additional information you might need is:

Type of packaging used
Description of goods
Shipping information
Date and terms of sale
2. Export packing list
The packing list is a detailed overview confirming the cargo mentioned on the commercial invoice. It also includes information on how the shipment has been packed and which marks and numbers are noted outside the boxes.

Customs officers in the US and the destination country use the packing list to check goods for inspection, and it can be used to help prepare the bill of lading (BoL).

3. Bill of lading (BoL)
The bill of lading is another essential document in international trade. It is the transportation contract that includes important details on the shipment.

If you ship with Maersk, you'll receive this detailed document from us. It acts as the legal document of title, which allows the person holding it to claim ownership of the cargo. This means that filling out your bill of lading accurately and completely is vital.

The bill of lading also acts as a contract of carriage, detailing the carrier's responsibilities to the parties involved in transporting the cargo. It is often informed by the Incoterms® you've agreed to.

Which import customs documents do you need for the United States?
Some documents used for exporting are also used for importing — for example, the bill of lading and commercial invoice. Here, we detail the additional documents you'll need to be aware of when importing goods into the US from another destination.

1. Importer Security Filing (ISF)
An Importer Security Filing (ISF, or "10+2") is a filing required by the US Customs and Border Protection (CBP) that document importing information and details. The ISF is required for shipments arriving in the US by the ocean and must be filed at least 24 hours before the vessel sailing from the last foreign port of departure to the US.

Importers who do not file the ISF are penalized with up to USD 5,000 for late filing.

2. Import license
The US Customs and Border Protection (CBP) is the administrative agency for importing and exporting to and from the US. It does not require you to have a license to import, but other agencies within the US government might require licenses for goods they regulate.

Equally, some countries around the world have restricted goods that can't be imported freely. These goods may also require a license to import.

What to do before setting up your import shipment to the US?
Make sure you have a US customs broker to handle your customs processing.
Set up a Customs Power of Attorney with that customs broker.
Discuss setting up a yearly continuous customs bond or setting up a single transaction bond. It is a good idea to speak with a customs broker to see which way works best for you, as a single transaction customs bond can be expensive and often costs more for one shipment than purchasing a continuous bond valid for 12 months.
Complete your ISF document and make sure it is 48 to 72 hours before vessel departure from the last foreign port to the US port.
Need more help with your customs documents and processes?
Many documents are standard across international trade, so if you've shipped before, some of them will be familiar.

New to shipping? When you sign up to Maersk and book a shipment, you can easily add customs services and get support from our network of customs experts around the world. They make shipping simple for businesses like yours.
}
"""

In [20]:
# Obtener una respuesta del modelo de Gemini utilizando el prompt definido de delimitadores
respuesta_delimitadores = get_gemini_completion(prompt=prompt_delimitadores)

# Mostrar el contenido de la respuesta como Markdown
display(Markdown(respuesta_delimitadores))

## Resumen y traducción de los documentos necesarios para la exportación a Estados Unidos:

**5 puntos clave:**

1. **Factura proforma o factura comercial:** Se necesita uno de estos dos documentos para exportar desde Estados Unidos. La factura proforma es un documento preliminar enviado al comprador antes del envío, mientras que la factura comercial es un documento completo que contiene información detallada sobre los productos, el fabricante, el origen, el destino y el código HTS.
2. **Lista de empaque:** Este documento proporciona una descripción detallada del contenido del envío, incluyendo información sobre cómo se ha empaquetado la mercancía y las marcas y números que se encuentran en las cajas.
3. **Conocimiento de embarque (BoL):** Este documento es el contrato de transporte que incluye detalles importantes sobre el envío, como la información del transportista, el destino y la fecha de entrega. Actúa como título de propiedad de la mercancía, permitiendo al poseedor reclamar la propiedad del envío.
4. **Declaración de Seguridad del Importador (ISF):** Este documento es obligatorio para los envíos que llegan a Estados Unidos por vía marítima y debe presentarse al menos 24 horas antes de que el barco zarpe del último puerto extranjero de partida hacia Estados Unidos.
5. **Licencia de importación:** Aunque no es obligatorio tener una licencia de importación para Estados Unidos, algunas agencias gubernamentales pueden requerirla para ciertos productos regulados. Algunos países también pueden tener restricciones a la importación de ciertos productos, lo que puede requerir una licencia. 


## **7. Define la longitud o formato de la respuesta esperada** <a name="define-la-longitud-o-formato-de-la-respuesta-esperada"></a>

In [21]:
prompt_tipo_respuesta = """
Extrae las entidades mencionadas en el siguiente texto acerca de destinos de exportación para la comida vegana en Colombia escrito por ProColombia. Primero extrae todos los nombres de comidas, preparación o frutas, luego extrae todos los nombres de países, luego extrae valores de exportación por bien o a nivel agregado.

Formato deseado:
Nombres de comidas, preparación o frutas: lista separada de empresas por comas
Nombres de países: lista separada de empresas por comas
Valores de exportación: lista separada de empresas por comas

Texto: {Destinos prometedores para la comida vegana colombiana
Foto: Shutterstock.
Cada vez hay más oferta en el mercado y cada vez hay más consumidores, inspirados, por ideas o por motivos de salud, en conseguir un estilo de vida relacionado con el veganismo.

Es justamente este escenario en el que se adentrarían los empresarios colombianos interesados en exportar comida vegana y es ese crecimiento en la oferta lo que lo convierte en un mercado prometedor que aspira a seguir creciendo en el futuro cercano.

El mercado vegano en América Latina
Latinoamérica es parte importante del mercado internacional vegano. Según la fundación Vegetarianos Hoy, en el continente hay varios países que se destacan por la oferta vegana que tienen sus supermercados, lo que los convierte en destinos en los que habitan cantidades importantes de consumidores de comida vegana y que, por lo mismo,   tienen las puertas abiertas a este tipo de bienes.

Entre estos, además de Colombia, se encuentran Argentina, Chile, México y Perú, ya que sus supermercados cuentan con una amplia oferta. A estos, los exportadores nacionales pueden llegar con los bienes de origen nacional a complementar esta oferta o como surtidores de la misma.

Esto también le da la oportunidad a los empresarios nacionales que integran ese sector productivo de llegar a un mercado internacional que se encuentra cercano a Colombia y que, en la mayoría de los casos, mantiene acuerdos comerciales vigentes con el país, lo que les da facilidades logísticas y beneficios arancelarios, en algunos casos.

Productos veganos congelados y refrigerados sobre una tabla para preparar platos veganos.

Un planeta vegano
Este panorama también se encuentra con facilidad en diversas partes del mundo. La oferta de alimentos veganos es amplia a nivel global y existen múltiples mercados a los que los exportadores colombianos pueden llegar.

Uno de los Objetivos de Desarrollo Sostenible planteados por las Naciones Unidas busca no solo la erradicación del hambre, sino que esta se logre con alimentos conseguidos a través de métodos sostenibles y esto le ha dado protagonismo a los bienes veganos dentro del mercado de los alimentos en el mundo, ya que en muchos casos su producción tiende a ser más sostenible que la producción de otro tipo de alimentos.

Además, según estudios como el Vegan Food Global Market Report 2023 del Research and Markets, se estima que el mercado de este tipo de bienes crezca un 12,4 % desde la actualidad hasta el 2027, mostrando una tendencia vegana en el mercado que se espera que se mantenga en los próximos años.

Por su parte, Happy Cow, una plataforma que se enfoca en la oferta de restaurantes veganos, logró identificar cuáles son las ciudades más amigables en el mundo con los alimentos de este tipo. Entre estas, se destacan Nueva York, en Estados Unidos Londres, en Inglaterra Berlín, en Alemania Melbourne, en Australia y Los Ángeles, también en Estados Unidos, siendo ese país el único con dos ciudades en el ranking y que, por lo mismo, se puede considerar un destino aún más prometedor que los otros que aparecen en la lista.

Además, Colombia mantiene diversos acuerdos comerciales con la mayoría de dichos destinos, lo que les da diferentes beneficios a los empresarios colombianos al momento de exportar sus bienes a esos países.

productos veganos en envases de vidrio preparados para el mercado mundial de productos veganos.

Otros posibles compradores
El país ya tiene una amplia experiencia en la exportación de bienes pertenecientes al sector de los Agroalimentos, lo que le ofrece a los exportadores de comida vegana un mercado en el que Colombia ya tiene una amplia experiencia en la exportación de bienes pertenecientes al sector de agroalimentos. Esto consolida al país como un proveedor de calidad ante el mundo, generando oportunidades para las industrias alimenticias crecientes en el país como la vegana.

Según datos del DANE  en el primer semestre del 2023 dicho sector exportó US$ FOB 6003,2 millones y, según Más Colombia, los principales destinos fueron Estados Unidos, Panamá y Países Bajos.

Colombia ya exporta comida vegana
Si usted es un empresario colombiano interesado en empezar a exportar alimentos veganos puede tener la tranquilidad de saber que no es el primero en dar este paso. En el país ya hay empresas que han decidido confiar en la rentabilidad de este tipo de bienes y han descubierto un mercado deseoso de adquirirlos.

Un ejemplo es el caso de El artesano, una empresa de Foodtech que ofrece alimentos saludables basados en proteína vegetal que ha llegado a países como Jamaica. Otro es el caso de Inducoal, una empresa colombiana que ofrece comida vegana proporcionada por agricultores del país. Finalmente, un último ejemplo es el de Pacifico Snacks, una empresa que fabrica snacks naturales de la más alta calidad y que comercia sus bienes en Estados Unidos.

Comidas veganas saludables y productos colombianos veganos distribuidos sobre una mesa.

Todas estas empresas ya encontraron el potencial de la comida vegana y han abierto las puertas de distintos mercados a los bienes nacionales que se producen dentro del sector agropecuario y de sectores veganos.

Además, Colombia está llamada, de acuerdo a la FAO, a ser una de las despensas del mundo (7 países ubicados en el trópico), por todos los atributos y la riqueza productiva del país, lo que le da un impulso a la industria de la comida vegana. No pierda la oportunidad de hacer parte del mercado de la comida vegana, uno que no para de crecer, y descubra el panorama de las exportaciones nacionales aquí.}
"""

In [22]:
# Obtener una respuesta del modelo de Gemini utilizando el prompt definido de tipo de respuesta
respuesta_tipo_respuesta = get_gemini_completion(prompt=prompt_tipo_respuesta)

# Mostrar el contenido de la respuesta como Markdown
display(Markdown(respuesta_tipo_respuesta))

## Entidades del texto sobre destinos de exportación para la comida vegana en Colombia:

**Nombres de comidas, preparación o frutas:**

* Comida vegana
* Alimentos veganos
* Bienes veganos
* Snacks naturales
* Proteína vegetal
* Alimentos saludables
* Productos veganos congelados y refrigerados
* Productos veganos en envases de vidrio
* Comidas veganas saludables
* Productos colombianos veganos

**Nombres de países:**

* Colombia
* Argentina
* Chile
* México
* Perú
* Estados Unidos
* Inglaterra
* Alemania
* Australia
* Jamaica
* Panamá
* Países Bajos

**Valores de exportación:**

* US$ FOB 6003,2 millones (Exportaciones del sector agroalimentario en el primer semestre de 2023) 


## **8. Proporciona texto de referencia** <a name="proporciona-texto-de-referencia"></a>

In [23]:
prompt_referencia = """
Utiliza el texto delimitado por triple comillas para responder preguntas. Si la respuesta no se puede encontrar en el texto, escribe "No pude encontrar una respuesta." El texto es una sección de la guía legal para hacer negocios en Colombia escrita por Procolombia.


{1.2. Acuerdos para evitar la Doble Tributación

Los ADT son instrumentos internacionales bilaterales o multilaterales encaminados a establecer reglas claras que eviten o mitiguen la doble tributación sobre ingresos o patrimonio, que, en circunstancias normales, estarían gravados con los mismos o similares tributos en dos o más jurisdicciones. Estos tratados se rigen por principios de derecho internacional público y también sirven para el fomento de la cooperación entre Estados a fin de combatir la evasión fiscal. Los ADT funcionan como un instrumento de atracción de la inversión extranjera y de promoción de la inversión de colombianos en el exterior, en la medida en que otorgan los siguientes beneficios:

i) Estabilidad tributaria sobre las condiciones mínimas de operación entre residentes fiscales de dos Estados.
ii) Reducción de la carga tributaria efectiva consolidada, vía acceso a tasas de retención reducidas.
iii) La posibilidad de exonerar un ingreso determinado de tributación, por lo general en el país de la fuente, en la medida en que no exista una presencia suficiente en dicho país por parte de la compañía del exterior.

Los ADT usualmente se limitan al impuesto sobre la renta y, en algunos casos, incluyen el impuesto al patrimonio; por lo general, impuestos indirectos
como el impuesto sobre las ventas (“IVA”) no están contemplados en estos instrumentos. Tampoco se incluyen impuestos territoriales como el impuesto de
industria y comercio (“ICA”).

En la evaluación realizada por los inversionistas al momento de seleccionar donde ejecutar sus proyectos de inversión, además de incluir la expectativa de retorno de la inversión y del riesgo, el impacto tributario juega un papel preponderante. Por ello, el régimen tributario, y en particular los ADTs, se convierten en elementos decisivos al momento de escoger el destino de su inversión.

Los inversionistas buscan evitar la doble tributación, cuyo problema principal surge por discrepancias en la aplicación de términos como el de residencia, fuente del ingreso, o de limitaciones para que los impuestos pagados en el extranjero por ingresos de fuente en el otro país, se permitan tomar como un crédito fiscal en el país de la residencia, o se trate el mismo como una renta exenta en dicho país de la residencia; lo que tiene como consecuencia que dos países graven el mismo ingreso durante un mismo periodo imponible y por la misma causa. Para resolver este problema, los ADTs establecen principios comunes de equidad en la distribución del ingreso, incentivando así las transacciones internacionales.

En este sentido, un indicador de la aplicación de principios comunes entre países y desarrollo tributario es el número de ADTs ratificados, ya que estos incrementan los niveles de retorno para el inversionista y fomentan la seguridad jurídica. De esta forma, una red de ADTs constituye una herramienta de atracción para la inversión extranjera.

Colombia ha seguido la tendencia mundial de adoptar como base de las negociaciones un modelo de Tratado que, en términos generales, podría ser el desarrollado por la  Organización para la Cooperación y el Desarrollo Económico (“OCDE”) o el de la Organización de las Naciones Unidas (“ONU”).
}

Pregunta:

1. ¿Por qué como inversionista debo buscar evitar la doble tributación?

2. ¿Cuál es el título del acuerdo de inversión vigente entre Colombia y México?

"""

In [24]:
# Obtener una respuesta del modelo de Gemini utilizando el prompt definido de texto de referencia
respuesta_texto_referencia = get_gemini_completion(prompt=prompt_referencia)

# Mostrar el contenido de la respuesta como Markdown
display(Markdown(respuesta_texto_referencia))

1.  Los inversionistas buscan evitar la doble tributación, cuyo problema principal surge por discrepancias en la aplicación de términos como el de residencia, fuente del ingreso, o de limitaciones para que los impuestos pagados en el extranjero por ingresos de fuente en el otro país, se permitan tomar como un crédito fiscal en el país de la residencia, o se trate el mismo como una renta exenta en dicho país de la residencia; lo que tiene como consecuencia que dos países graven el mismo ingreso durante un mismo periodo imponible y por la misma causa. 

2. No pude encontrar una respuesta. 


## **9. Divide la tarea en pasos individuales** <a name="divide-la-tarea-en-pasos-individuales"></a>

In [25]:
prompt_tareas = """
Realiza una investigación de mercado para exportar productos textiles de Colombia a la región de Ámerica Latina siguiendo estos pasos:

1. Identifica el producto que se desea exportar.
2. Determina los países potenciales para la exportación.
3. Investiga las regulaciones de importación del país destino.
4. Analiza la demanda y competencia en el mercado objetivo.
5. Recopila datos sobre aranceles y otros costos asociados.
6. Evalúa las preferencias de los consumidores en el país objetivo.
7. Desarrolla una estrategia de entrada al mercado (e.g., distribuidores locales, tiendas propias).
8. Crea un plan de marketing adaptado al nuevo mercado.

Tu respuesta no debe tener más de 500 palabras
"""

In [26]:
# Obtener una respuesta del modelo de Gemini utilizando el prompt definido de tareas
respuesta_tareas = get_gemini_completion(prompt=prompt_tareas)

# Mostrar el contenido de la respuesta como Markdown
display(Markdown(respuesta_tareas))

## Investigación de mercado para exportar productos textiles de Colombia a América Latina:

**1. Producto:** Ropa de algodón de alta calidad para niños, con diseños coloridos y originales.

**2. Países potenciales:** México, Perú, Chile, Argentina, Ecuador. Estos países presentan un mercado de ropa infantil en crecimiento y una afinidad por la moda colombiana.

**3. Regulaciones de importación:** Cada país tiene sus propias regulaciones, incluyendo requisitos de etiquetado, normas de seguridad y aranceles. Es crucial investigar las regulaciones específicas de cada país destino antes de exportar.

**4. Demanda y competencia:** Existe una alta demanda de ropa infantil de calidad en la región, especialmente por productos con diseños únicos y sostenibles. La competencia es diversa, incluyendo marcas internacionales y locales.

**5. Aranceles y costos:** Los aranceles varían según el país y el tipo de producto. Es necesario investigar los aranceles específicos para la ropa de algodón infantil y otros costos asociados como transporte, seguros y logística.

**6. Preferencias de los consumidores:** Los consumidores latinoamericanos buscan ropa infantil cómoda, durable, con diseños atractivos y que reflejen la cultura local. La sostenibilidad y la responsabilidad social también son factores importantes.

**7. Estrategia de entrada al mercado:** 
* **Distribuidores locales:** Se puede colaborar con tiendas de ropa infantil existentes o plataformas online de comercio electrónico.
* **Tiendas propias:** Abrir tiendas propias en ciudades clave puede ser una opción a largo plazo, pero requiere una mayor inversión.

**8. Plan de marketing:**
* **Marketing digital:** Utilizar redes sociales, publicidad online y SEO para llegar al público objetivo.
* **Colaboraciones:** Colaborar con influencers locales y bloggers de moda infantil.
* **Eventos:** Participar en ferias comerciales y eventos de moda infantil.
* **Comunicación:** Enfatizar la calidad, diseño, sostenibilidad y origen colombiano de la ropa.

**Conclusión:** Exportar productos textiles de Colombia a América Latina presenta un potencial significativo. Es crucial realizar una investigación exhaustiva del mercado, cumplir con las regulaciones de importación, desarrollar una estrategia de entrada adecuada y adaptar el plan de marketing al público objetivo. 


# **Caso de estudio: Exportación de productos de café a China** <a name="caso-de-estudio-exportación-de-productos-de-café-a-china"></a>

### **Contexto de negocio**

Global Colombian Foods S.A es una empresa colombiana que exporta frutas tropicales y comida semi procesada a Europa y Norteamérica. Recientemente, han decidido expandir su mercado de productos de café al continente asiático, específicamente a China. Necesitan realizar una investigación de mercado exhaustiva para entender la actualidad de la economía de dicho país y regulaciones de importación.

### **Enfoque analítico:**
Utilizaremos Python y la Gemini para realizar una investigación de mercado asistida por IA. Nos centraremos en cómo formular prompts efectivos para obtener información relevante y cómo interpretar y validar los resultados obtenidos.

## **10. Traducción y resumen de videos en otro idioma** <a name="traducción-y-resumen-de-videos-en-otro-idioma"></a>

En nuestro ejercicio encontramos dos videos de actualidad económica de China del canal de noticias DW, sin embargo, los videos tienen una combinación de chino e inglés. Nuestra tarea consistirá en usar las transcripciones de los videos en inglés y chino para generar un informe.

Videos:
1. ¿Hacia dónde irá la economía de China en 2024? Lo entenderás con solo mirar DW - 2024中國經濟將要走向何方？| DW一看你就懂: https://www.youtube.com/watch?v=ckU5lLLYZTs
2. ¿Algún día la economía de China alcanzará a la de Estados Unidos? | DW Lo entenderás con solo mirarlo - 中國經濟還會有趕超美國的那一天嗎？| DW 一看你就懂: https://www.youtube.com/watch?v=U_ULTsDU4aM

Luego descargamos los subtitulos del video en: https://downsub.com/

**Video 1: ¿Hacia dónde irá la economía de China en 2024?**

In [27]:
# Definir los nombres de los archivos
file_1 = '[Chinese Traditional] China 2024.txt'
file_2 = '[English] China 2024.txt'

# Leer el archivo de texto en chino tradicional
with open(file_1, 'r', encoding='utf-8') as file:
    transcripciones_chino_video_1 = file.read()

# Leer el archivo de texto en inglés
with open(file_2, 'r', encoding='utf-8') as file:
    transcripciones_ingles_video_1 = file.read()

In [28]:
# Mostrar el contenido de la transcripción en chino
print(transcripciones_chino_video_1)

我記得是2019年的時候

說起中國經濟

有一句話在當時中國的社群媒體上非常流行

它是這麼說的:

2019年可能會是過去十年裡最糟糕的一年

但卻是未來十年最好的一年

當時聽到這句話時

新冠在中國官方的眼中還只是一個謠言

俄羅斯還沒有入侵烏克蘭

哈馬斯和以色列還沒有展開新一輪的你死我活

中國經濟也還沒有陷入房地產市場和A股大步走下坡路

地方政府公務員、銀行 職員發不出工資

通貨緊縮和內需疲軟纏身的境地

然後的幾年中

每逢新年之際

都不得不佩服當初說出這句話的人的遠見和智慧

當然你還是可以說

他只是湊巧說中

你當然可以說

悲觀者正確

樂觀者前進

但時間來到了2024年初

我們可以看到的是

中國經濟正面臨前所未有的挑戰

2023年雖然已經夠艱難

但現在你又可以聽到有人說

過去的一年

會是未來10年最好的一年

中國經濟未來到底路在何方？

悲觀者會再次正確

還是說未來會給悲觀者piapia打臉

撫慰負重前行的樂觀者們？

歡迎您關注德國之聲DW在新的一年裡帶給您的「一看你就懂」

中國經濟在2023年末

2024年初的狀態怎樣

未來會如何？

我們先來看看歐洲方面的一些聲音是怎麼說的？

再看看來自中國本地的聲音是怎麼說的？

我們對2024年的整體判斷

其實是會比2023年更積極樂觀一些

同樣還會有很多挑戰和考驗

中國經濟本身我們認為還是很有韌性的

我們覺得到了24年的時候

有前面的一些積累

走出疫情的第二年應該會有更多的機會

中國經濟目前的現況目前是穩定發展

相對消費上面保守一些

但是經濟增長和進出口問題不大

所以24年應該是穩中向上的過程

從我們訪談的各路專家和分析人士的表態中

能夠清楚的看到一個這樣的趨勢

對於我們此次節目的大主題

2024中國經濟將要走向何方？的判斷

中國國內的聲音和國外的聲音

以及一般民眾的聲音和社會精英層的聲音

宛如來自兩個不同的星球

一方面無論是在歐洲生活工作的華人或學者專家

談到對未來中國經濟走勢的判斷大都持相對悲觀的態度

而如果你聆聽中國國內的聲音

無論是一般民眾或智庫機構的專家們

對中國經濟未來的發展前景充滿了信心

有一種「利空出盡是利好」的信念

同時也能觀察到的一個有趣的現像是

越是社會中的一般民眾

對中國未來的發展越更有

In [29]:
# Mostrar el contenido de la transcripción en inglés
print(transcripciones_ingles_video_1)

I remember that in 2019, when talking about the Chinese economy, there was a saying that was very popular on Chinese social media at the time. It said: 2019 may be the worst year in the past ten years, but it will be the worst year in the next ten years. The best year was when I heard this sentence. In the eyes of Chinese officials, the new crown was just a rumor. Russia had not yet invaded Ukraine. Hamas and Israel had not yet launched a new round of life-and-death struggle. The Chinese economy had not yet fallen into the real estate market and A. The stock market has been on a sharp decline. Local government civil servants and bank employees have been unable to pay their wages. We have been plagued by deflation and weak domestic demand. In the following years, every New Year's occasion has to be admired for the foresight and wisdom of the person who originally said these words. Of course, You can still say that he just happened to be right. Of course you can say that the pessimists a

In [30]:
# Crear un prompt para obtener insights valiosos del reportaje
prompt_china_2024 = f"""
Hola, somos de nuevo nosotros.

Estamos realizando un estudio de mercado sobre el mercado chino y la posibilidad de exportar productos alimenticios a su mercado. El primer paso en nuestra investigación es analizar la actualidad económica de China.

Para ello, encontramos un video del canal de noticias DW China titulado "¿Hacia dónde irá la economía de China en 2024? Lo entenderás con solo mirar DW".

El video combina diálogos de expertos económicos en Chino e inglés. Con una herramienta externa descargamos los subtítulos o transcripciones del video en ambos idiomas y ahora queremos obtener información clara y relevante de ellos.

Para ello debes realizar la siguiente tarea:

1. Objetivo: traducir el contenido de las transcripciones de chino a español y de inglés a español y crear un informe con la información más relevante del video.

Aspectos que debes tener en cuenta:
    1. El resumen debe tener coherencia y cohesión y brindar un panorama claro sobre la actualidad y perspectivas económicas de China.
    2. Agrega información relevante que pueda complementar lo dicho en el video.

A continuación te compartimos los textos de las transcripciones en delimitados por []:

Transcripción en Chino: [{transcripciones_chino_video_1}]

Transcripción en Inglés: [{transcripciones_ingles_video_1}]
"""

# Limpiar el prompt de espacios o líneas en blanco
prompt_china_2024 = clean_text(prompt_china_2024)

# Mostrar el contenido del prompt limpio
print(prompt_china_2024)

Hola, somos de nuevo nosotros.
Estamos realizando un estudio de mercado sobre el mercado chino y la posibilidad de exportar productos alimenticios a su mercado. El primer paso en nuestra investigación es analizar la actualidad económica de China.
Para ello, encontramos un video del canal de noticias DW China titulado "¿Hacia dónde irá la economía de China en 2024? Lo entenderás con solo mirar DW".
El video combina diálogos de expertos económicos en Chino e inglés. Con una herramienta externa descargamos los subtítulos o transcripciones del video en ambos idiomas y ahora queremos obtener información clara y relevante de ellos.
Para ello debes realizar la siguiente tarea:
1. Objetivo: traducir el contenido de las transcripciones de chino a español y de inglés a español y crear un informe con la información más relevante del video.
Aspectos que debes tener en cuenta:
1. El resumen debe tener coherencia y cohesión y brindar un panorama claro sobre la actualidad y perspectivas económicas de

In [31]:
# Ejecutar el prompt con OpenAI
respuesta_openai_video_1 = get_completion_open_ai(prompt=prompt_china_2024)

# Mostrar el contenido como Markdown
display(Markdown(respuesta_openai_video_1))

### Informe sobre la Actualidad y Perspectivas Económicas de China (2024)

#### Introducción
La economía china se encuentra en un momento crítico a inicios de 2024, enfrentando desafíos sin precedentes. Este informe sintetiza las opiniones y análisis presentados en el video del canal DW China, titulado "¿Hacia dónde irá la economía de China en 2024?", complementando la información con datos relevantes sobre la situación económica actual y las proyecciones futuras.

#### Contexto Actual
En 2019, se popularizó en las redes sociales chinas una frase que predecía que ese año sería el peor de la última década, pero también el mejor de la próxima. A medida que se acercaba 2024, la economía china ha enfrentado múltiples crisis, incluyendo la caída del mercado inmobiliario, problemas de deflación y una demanda interna débil. A finales de 2023, la situación económica se percibe como inestable, con un crecimiento que se espera sea más bajo en comparación con años anteriores.

#### Perspectivas para 2024
1. **Optimismo y Desafíos**: A pesar de los retos, hay un optimismo moderado sobre la economía china en 2024. Expertos y analistas coinciden en que, aunque habrá desafíos, la economía tiene una notable resiliencia. Se espera que el crecimiento del PIB sea del 5% en 2024, aunque esto es inferior al 5.4% proyectado para 2023.

2. **Diferencias de Opinión**: Existe una notable discrepancia entre las percepciones de los expertos en el extranjero y las voces dentro de China. Mientras que muchos académicos y profesionales chinos en el extranjero son pesimistas sobre el futuro económico, los ciudadanos y expertos locales muestran una confianza considerable en la recuperación económica.

3. **Proyecciones del FMI**: El Fondo Monetario Internacional (FMI) ha ajustado sus previsiones, aumentando la expectativa de crecimiento del PIB para 2023 y 2024. Sin embargo, las proyecciones para los años siguientes (2025-2028) muestran una tendencia a la baja, con tasas de crecimiento que podrían caer por debajo del 4%.

4. **Factores de Crecimiento**: Se anticipa que la economía china se beneficiará de sectores emergentes como la energía nueva, la inteligencia artificial y la digitalización. La transición hacia estos sectores podría ser clave para el crecimiento futuro, especialmente si se compara con el papel que desempeñó el sector inmobiliario en el pasado.

5. **Legitimidad del Gobierno**: La legitimidad del Partido Comunista Chino está fuertemente ligada al crecimiento económico. La percepción de que el gobierno debe mejorar las condiciones de vida de la población es fundamental para su aceptación. Sin embargo, la creciente insatisfacción social podría representar un riesgo si la economía no muestra signos de recuperación.

#### Conclusiones
La economía china se encuentra en una encrucijada. A pesar de las proyecciones de crecimiento moderado y los desafíos persistentes, hay un consenso sobre la resiliencia de la economía y la posibilidad de recuperación a través de la innovación y la adaptación a nuevas realidades económicas. Sin embargo, la capacidad del gobierno para manejar la insatisfacción social y mantener la legitimidad será crucial en los próximos años.

#### Recomendaciones para la Exportación de Productos Alimenticios
1. **Investigación de Mercado**: Es esencial realizar un análisis detallado del comportamiento del consumidor chino y las tendencias de consumo, especialmente en el contexto post-pandemia.
2. **Adaptación a Normativas**: Asegurarse de cumplir con las regulaciones y estándares de calidad que rigen la importación de productos alimenticios en China.
3. **Colaboración Local**: Considerar asociaciones con empresas locales para facilitar la entrada al mercado y mejorar la comprensión de las dinámicas del consumidor.

Este informe proporciona un panorama claro sobre la situación económica actual de China y sus perspectivas para 2024, ofreciendo una base sólida para la toma de decisiones en el ámbito de la exportación de productos alimenticios.

In [32]:
# Ejecutar el prompt con Gemini
respuesta_gemini_video_1 = get_gemini_completion(prompt=prompt_china_2024)

# Mostrar el contenido de la respuesta como Markdown
display(Markdown(respuesta_gemini_video_1))

## Informe: Perspectivas Económicas de China en 2024

Este informe analiza las perspectivas económicas de China en 2024, basándose en las transcripciones del video de DW China "¿Hacia dónde irá la economía de China en 2024? Lo entenderás con solo mirar DW". 

**Contexto:**

* El video presenta un panorama complejo, con opiniones divergentes sobre el futuro económico de China. 
* Se destaca la incertidumbre que rodea la economía china, marcada por la recuperación post-pandemia, la guerra en Ucrania y las tensiones geopolíticas.
* Se menciona la frase popular en China en 2019: "2019 puede ser el peor año de la última década, pero será el mejor de la próxima".

**Perspectivas:**

**1. Desafíos y Oportunidades:**

* **Desafíos:**
    * La economía china enfrenta desafíos sin precedentes, incluyendo la recuperación post-pandemia, la desaceleración global y las tensiones geopolíticas.
    * Se menciona la caída del mercado inmobiliario, la disminución del consumo y la dificultad para pagar salarios en algunas áreas.
* **Oportunidades:**
    * Se espera que la economía china sea más resiliente y que haya más oportunidades en 2024, con un crecimiento estable y un aumento de las exportaciones.
    * Se destaca la importancia de la tecnología como motor de crecimiento, con especial atención a la energía renovable, la digitalización y la innovación.

**2. Divergencia de Opiniones:**

* **Opiniones Pesimistas:**
    * Expertos internacionales, especialmente en Europa, muestran un panorama más pesimista, con proyecciones de crecimiento más bajas para los próximos años.
    * El Fondo Monetario Internacional (FMI) proyecta un crecimiento del 5.4% para 2023 y un 4.6% para 2024, con tasas aún más bajas para los siguientes años.
* **Opiniones Optimistas:**
    * Las voces dentro de China, incluyendo expertos y la población en general, son más optimistas, con una confianza en la capacidad de la economía para recuperarse y crecer.
    * Se destaca la importancia de la innovación tecnológica y la confianza en que China podrá superar los desafíos y lograr un crecimiento sostenido.
    * La visión optimista se basa en la idea de que "el mal ya pasó" y que China está bien posicionada para aprovechar las oportunidades que se presentan.

**3. La Importancia del Crecimiento Económico:**

* El crecimiento económico es crucial para China, tanto para la estabilidad política como para el bienestar de la población.
* Se menciona la importancia del crecimiento para la legitimidad del Partido Comunista, que se basa en la promesa de una vida mejor para los ciudadanos.
* Se destaca la preocupación por la posibilidad de que la población se vuelva más crítica ante una desaceleración económica, lo que podría generar inestabilidad social.

**4. El Papel del Gobierno:**

* El gobierno chino está tomando medidas para impulsar la economía, incluyendo políticas de estímulo para el sector inmobiliario, la apertura a la inversión extranjera y la emisión de deuda para apoyar a las regiones con dificultades.
* Se debate si estas medidas son suficientes para abordar los problemas estructurales de la economía china, o si solo son soluciones a corto plazo.

**5. La Resiliencia de China:**

* A pesar de los desafíos, se destaca la capacidad de China para superar las crisis y continuar su crecimiento económico.
* Se menciona la importancia de la economía china para el mundo, y su capacidad para atraer inversiones y generar oportunidades.

**Conclusión:**

El futuro económico de China en 2024 es incierto, con opiniones divergentes entre expertos nacionales e internacionales. Si bien existen desafíos importantes, también hay oportunidades para el crecimiento y la innovación. El gobierno chino está tomando medidas para impulsar la economía, pero aún queda por ver si estas serán suficientes para garantizar un crecimiento sostenido. La capacidad de China para superar las crisis y su papel crucial en la economía global sugieren que la historia de crecimiento del país aún no ha terminado. 


In [33]:
# Logramos obtener el panorama económico con ambos modelos, pero investiguemos cuántos tokens usamos en cada consulta
# Para los modelos de OpenAI primero confirmemos la codificación que emplea el modelo que estamos usando:
encoding = tiktoken.encoding_for_model('gpt-4o-mini')
encoding

<Encoding 'o200k_base'>

In [34]:
# Transformar el prompt en tokens utilizando la codificación obtenida
prompt_tokens = encoding.encode(prompt_china_2024)

# Mostrar los tokens generados
prompt_tokens

[49864,
 11,
 42665,
 334,
 18094,
 31448,
 558,
 93378,
 103781,
 537,
 40647,
 334,
 17823,
 4844,
 650,
 17823,
 97207,
 342,
 557,
 48996,
 334,
 11766,
 277,
 22809,
 15293,
 13400,
 261,
 593,
 17823,
 13,
 3241,
 13240,
 33250,
 469,
 19972,
 43096,
 878,
 92098,
 557,
 80557,
 62389,
 334,
 8114,
 558,
 15115,
 28949,
 11,
 75132,
 537,
 3823,
 1083,
 31477,
 334,
 74186,
 67939,
 8114,
 7831,
 56434,
 392,
 11986,
 39,
 21820,
 80203,
 50869,
 557,
 67080,
 334,
 8114,
 469,
 220,
 1323,
 19,
 30,
 8786,
 37412,
 2294,
 406,
 9947,
 108487,
 67939,
 26880,
 4422,
 3823,
 76745,
 112786,
 70972,
 334,
 85951,
 105136,
 469,
 1036,
 2081,
 319,
 69173,
 13,
 1532,
 1969,
 72857,
 101238,
 159364,
 4064,
 1403,
 25091,
 56921,
 293,
 1643,
 146681,
 1083,
 3823,
 469,
 40912,
 131921,
 342,
 20943,
 55224,
 35322,
 19675,
 64691,
 342,
 68055,
 334,
 22884,
 558,
 15115,
 28949,
 69610,
 19250,
 557,
 36487,
 78678,
 734,
 16,
 13,
 115602,
 4330,
 25,
 142215,
 380,
 650,
 44442

In [42]:
# Función para contar el número de tokens en una cadena de texto
def num_tokens_from_string(string: str, encoding_name: str):
    """
    Retorna el número de tokens en una cadena de texto.

    Parámetros:
    string (str): La cadena de texto para la cual se desea contar los tokens.
    encoding_name (str): El nombre de la codificación a utilizar para el conteo de tokens.

    Retorna:
    int: El número de tokens en la cadena de texto.
    """
    # Obtener la codificación basada en el nombre de la codificación proporcionada
    encoding = tiktoken.get_encoding(encoding_name)

    # Codificar la cadena de texto y contar el número de tokens
    num_tokens = len(encoding.encode(string))

    # Retornar el número de tokens
    return num_tokens

In [43]:
# Número de tokens para el modelo de OpenAI
tokens_chatgpt = num_tokens_from_string(string=prompt_china_2024, encoding_name='o200k_base')

In [44]:
# Crear el modelo de Gemini
model_video_1 = genai.GenerativeModel("gemini-1.5-flash")

# Contar el número de tokens en el prompt utilizando el modelo de Gemini
tokens_gemini = model_video_1.count_tokens(contents=prompt_china_2024)

In [45]:
# Comparemos
print(f"""
    - El modelo gpt4-mini consume {str(tokens_chatgpt).strip()} para el prompt de resumen de videos
    - El modelo gemini 1.5 flash consume {str(tokens_gemini).strip()} para el prompt de resumen de videos
""")


    - El modelo gpt4-mini consume 6164 para el prompt de resumen de videos
    - El modelo gemini 1.5 flash consume total_tokens: 5652 para el prompt de resumen de videos



**Video 2: ¿Algún día la economía de China alcanzará a la de Estados Unidos?**

In [46]:
# Definir los nombres de los archivos
file_1 = '[Chinese Traditional] China vs Estados Unidos.txt'
file_2 = '[English] China vs Estados Unidos.txt'

# Leer el archivo de texto en chino tradicional
with open(file_1, 'r', encoding='utf-8') as file:
    transcripciones_chino_video_2 = file.read()

# Leer el archivo de texto en inglés
with open(file_2, 'r', encoding='utf-8') as file:
    transcripciones_ingles_video_2 = file.read()


In [47]:
# Mostrar el contenido de la transcripción en chino
print(transcripciones_chino_video_2)

它書寫了我們這個時代最引人注目的經濟發展成功故事

然而現在 它的光環正在逐漸黯淡

在過去的20年裡 中國的經濟總量在世界上的排名不斷攀升

它實現了人類有記載以來主要經濟體中速度最快的持續增長

如今中國的經濟規模位列世界第二 僅次於美國

中國的快速崛起引發了各種關於它很快會成爲世界最大經濟體的猜測

如今中國經濟發展的前景烏雲密佈

20年以來 中國經濟增長速度將會首次落後於亞洲其它國家

隨著中國經濟增長乏力 很多專家開始重新思考

這個全球第二大經濟體究竟什麽時候才會超過美國

或者它是否還能有超過美國的那一天

各位好 歡迎來到DW德國之聲中文YouTube頻道

本期「一看你就懂」我們來一起探討中國經濟增速放緩的背景

我們也會一起來審視哪些國際外部因素威脅到中國的經濟增長

最後我們會一起來探討這個核心問題：中國是否仍然有望成爲世界最大經濟體？

首先讓我們來看看告別「清零」之後的中國經濟增長前景如何

2010年中國經濟增長率達到了令人矚目的10.6%

從那之後 增長速度開始放緩

根據國際貨幣基金組織的預測

中國經濟增速減緩的趨勢還會繼續——並且持續到疫情結束之後

根據預測 未來數年中國GDP的增幅也會徘徊在5%以下的水平

不過也有一些預測相對更樂觀一點

對中國經濟增長起到巨大影響的因素就是「清零政策」

嚴格的防疫管控在過去三年裡主導了中國人的生活

2022年 世界大多數國家都開始重返正常生活的同時 中國卻收緊了管控

嚴格的防疫措施遏制了國内消費 令工廠陷入停產企業面臨生存困境

同時也引發了數十年來最大規模的反政府抗議活動

2022年12月 中國逐漸告別「清零政策」 放鬆對經濟的管控

但很多分析人士認爲 「清零」時代的結束並不意味著中國經濟能夠重拾過去的高增長

抛開一些短期性議題 要想深層次瞭解阻礙中國經濟增長的潛在因素

我們有必要首先回顧一下 中國是如何走上這條經濟增長快車道的

半個多世紀前的中國深陷文革的「十年浩劫」之中

整個國家處於政治 經濟和社會的混亂狀態

直到毛澤東去世 鄧小平逐漸掌握權力之後 局面發生了徹底的改變

改革開放給中國經濟帶來了翻天覆地的變化

城市規模不斷擴大 製造業蓬勃發展

人們的創業精神空前高漲

中國的中產階層在不斷壯大

鄧小平的改革喚醒了一個沉睡的經濟超級大國

隨著逐漸打開

In [48]:
# Mostrar el contenido de la transcripción en inglés
print(transcripciones_ingles_video_2)

It has written the most compelling economic development success story of our time. However, its halo is gradually dimming. In the past 20 years, China's economic aggregate has continued to rise in the world's rankings. It has achieved the major economic development in human history. The fastest sustained growth in the world. Today, China’s economy ranks second in the world after the United States. China’s rapid rise has triggered speculation that it will soon become the world’s largest economy. Today, China’s economic development prospects are clouded. For the first time in 20 years, China's economic growth will lag behind other Asian countries. With China's economic growth sluggish, many experts are beginning to rethink when the world's second largest economy will surpass the United States or whether it can ever surpass the United States. Hello everyone, welcome to DW Deutsche Welle Chinese YouTube channel. In this issue of "You'll Understand Once You See It", we will discuss the back

In [49]:
# Crear un prompt para obtener insights valiosos del reportaje
prompt_china_estados_unidos = f"""
Para la segunda parte de nuestro ejercicio de análisis económico,  encontramos un video del canal de noticias DW China titulado "¿Algún día la economía de China alcanzará a la de Estados Unidos? | DW Lo entenderás con solo mirarlo".

De igual forma que el primer video, este video combina diálogos de expertos económicos en Chino e inglés y también descargamos los subtítulos o transcripciones del video en ambos idiomas.

Ahora debes realizar la siguiente tarea:

1. Objetivo: traducir el contenido de las transcripciones de chino a español y de inglés a español y crear un informe con la información más relevante del video.

Aspectos que debes tener en cuenta:
    1. El resumen debe tener coherencia y cohesión y brindar un panorama claro sobre la competencia entre China y Estados Unidos.
    2. Agrega información relevante que pueda complementar lo dicho en el video.

A continuación te compartimos los textos de las transcripciones en delimitados por []:

Transcripción en Chino: [{transcripciones_chino_video_2}]

Transcripción en Inglés: [{transcripciones_ingles_video_2}]
"""

# Limpiar el prompt de espacios o líneas en blanco
prompt_china_estados_unidos = clean_text(prompt_china_estados_unidos)

# Mostrar el contenido del prompt limpio
print(prompt_china_estados_unidos)

Para la segunda parte de nuestro ejercicio de análisis económico,  encontramos un video del canal de noticias DW China titulado "¿Algún día la economía de China alcanzará a la de Estados Unidos? | DW Lo entenderás con solo mirarlo".
De igual forma que el primer video, este video combina diálogos de expertos económicos en Chino e inglés y también descargamos los subtítulos o transcripciones del video en ambos idiomas.
Ahora debes realizar la siguiente tarea:
1. Objetivo: traducir el contenido de las transcripciones de chino a español y de inglés a español y crear un informe con la información más relevante del video.
Aspectos que debes tener en cuenta:
1. El resumen debe tener coherencia y cohesión y brindar un panorama claro sobre la competencia entre China y Estados Unidos.
2. Agrega información relevante que pueda complementar lo dicho en el video.
A continuación te compartimos los textos de las transcripciones en delimitados por []:
Transcripción en Chino: [它書寫了我們這個時代最引人注目的經濟發展成功故事


In [50]:
# Ejecutar el prompt con Gemini
respuesta_gemini_video_2 = get_gemini_completion(prompt=prompt_china_estados_unidos)

# Mostrar el contenido de la respuesta como Markdown
display(Markdown(respuesta_gemini_video_2))

## Informe: ¿Algún día la economía de China alcanzará a la de Estados Unidos?

**Introducción:**

El video de DW China explora la posibilidad de que la economía china supere a la estadounidense, analizando los factores que podrían impulsar o frenar este escenario. Se destaca que, a pesar del crecimiento económico notable de China en las últimas décadas, su ritmo se ha desacelerado y enfrenta desafíos internos y externos.

**Desafíos internos:**

* **Desaceleración económica:** El crecimiento económico de China ha disminuido significativamente desde su pico en 2010. El FMI predice que seguirá desacelerándose, con un crecimiento por debajo del 5% en los próximos años.
* **"Política de Cero COVID":** Las estrictas medidas de control de la pandemia, aunque exitosas en términos de salud pública, tuvieron un impacto negativo en la economía, limitando el consumo y la producción.
* **Deuda pública:** China enfrenta un problema de deuda creciente, principalmente a nivel local, producto de una inversión excesiva en infraestructura y la especulación en el mercado inmobiliario. La crisis de Evergrande es un ejemplo de los riesgos que esto conlleva.
* **Crisis inmobiliaria:** El mercado inmobiliario chino está inflado y enfrenta un riesgo de colapso. La construcción de "ciudades fantasma" con viviendas vacías evidencia la sobreoferta y la burbuja especulativa.
* **Baja tasa de natalidad:** La tasa de natalidad en China ha caído a niveles históricamente bajos, lo que genera preocupación por la disminución de la fuerza laboral y el envejecimiento de la población.
* **Consumo interno:** A pesar del crecimiento de la clase media, el consumo interno no ha logrado impulsar la economía como se esperaba, debido a la alta tasa de ahorro y la incertidumbre económica.

**Desafíos externos:**

* **Desglobalización:** La tensión entre Estados Unidos y China, junto con la pandemia, está impulsando una desglobalización, lo que dificulta el acceso de China a mercados y recursos globales.
* **Disminución de la demanda:** La disminución de la demanda global, especialmente en Estados Unidos, afecta las exportaciones chinas, un motor clave de su economía.
* **Competencia estratégica:** La competencia estratégica entre Estados Unidos y China está creando un ambiente geopolítico complejo, con posibles consecuencias negativas para la economía china.

**Posibles escenarios:**

* **Superación de la economía estadounidense:** Algunos expertos creen que China aún tiene potencial para superar a Estados Unidos, pero requiere una transformación económica hacia un modelo de crecimiento impulsado por el consumo y la innovación.
* **Estancamiento:** Otros expertos advierten que la desaceleración económica, la deuda pública y los desafíos demográficos podrían impedir que China alcance a Estados Unidos, llevando a un estancamiento o incluso a una crisis.

**Conclusión:**

El futuro de la economía china es incierto. Superar a Estados Unidos como la economía más grande del mundo es un objetivo ambicioso que enfrenta desafíos internos y externos. El éxito de China dependerá de su capacidad para reformar su modelo económico, controlar la deuda, estimular el consumo interno y adaptarse a un entorno global cambiante.

**Información adicional:**

* El video destaca la importancia de la "reforma y apertura" de Deng Xiaoping como motor del crecimiento económico de China.
* Se menciona el ingreso de China a la Organización Mundial del Comercio (OMC) en 2001 como un factor clave en su expansión económica.
* Se hace referencia a la "política de un solo hijo", implementada en China entre 1980 y 2015, como una de las causas de la disminución de la población.

**Reflexiones:**

* La competencia económica entre Estados Unidos y China es un tema complejo con implicaciones globales.
* La economía china se encuentra en un punto de inflexión, donde deberá afrontar los desafíos de la desaceleración económica, la deuda pública y el cambio demográfico.
* La capacidad de China para adaptarse y transformarse será crucial para su futuro económico.


**Combinar informes de ambos videos**

In [51]:
# Función para crear un prompt para un informe basado en resultados previos
def crear_prompt_informe(resultados_prompts: list):
    """
    Crea un prompt para un modelo de LLM que le indica que, basándose en sus respuestas anteriores,
    relaciones y haga un informe cohesionado y coherente.

    Parámetros:
    resultados_prompts (list): Una lista que contiene los resultados de los prompts anteriores.

    Retorna:
    str: El prompt generado.
    """
    # Instrucción inicial para el modelo
    prompt = (
        """En base a las siguientes respuestas previas, por favor elabora un informe que sintetice la información proporcionada.
        Asegúrate de mantener la claridad y la cohesión en el informe.\n\n"""
    )

    # Agregar cada resultado de los prompts anteriores al prompt final
    for i, resultado in enumerate(resultados_prompts, 1):
        prompt += f"Respuesta {i}:\n{resultado}\n\n"

    # Instrucción final para elaborar el informe
    prompt += "Elabora el informe a continuación:"

    return prompt

In [52]:
# Crear lista a partir de las respuestas anteriores
resultados_gemini = [respuesta_gemini_video_1, respuesta_gemini_video_2]

# Llamar a la función para crear el prompt
prompt_generado_gemini = crear_prompt_informe(resultados_gemini)

# Mostrar el prompt generado
print(prompt_generado_gemini)

En base a las siguientes respuestas previas, por favor elabora un informe que sintetice la información proporcionada. 
        Asegúrate de mantener la claridad y la cohesión en el informe.

Respuesta 1:
## Informe: Perspectivas Económicas de China en 2024

Este informe analiza las perspectivas económicas de China en 2024, basándose en las transcripciones del video de DW China "¿Hacia dónde irá la economía de China en 2024? Lo entenderás con solo mirar DW". 

**Contexto:**

* El video presenta un panorama complejo, con opiniones divergentes sobre el futuro económico de China. 
* Se destaca la incertidumbre que rodea la economía china, marcada por la recuperación post-pandemia, la guerra en Ucrania y las tensiones geopolíticas.
* Se menciona la frase popular en China en 2019: "2019 puede ser el peor año de la última década, pero será el mejor de la próxima".

**Perspectivas:**

**1. Desafíos y Oportunidades:**

* **Desafíos:**
    * La economía china enfrenta desafíos sin precedentes, i

In [53]:
# Ejecutar el prompt con Gemini
respuesta_resumen_gemini = get_gemini_completion(prompt=prompt_generado_gemini)

# Mostrar el contenido de la respuesta como Markdown
display(Markdown(respuesta_resumen_gemini))

## Informe: Perspectivas Económicas de China en 2024 y su Camino hacia la Superación de Estados Unidos

Este informe sintetiza la información proporcionada en dos respuestas previas sobre la economía china, analizando sus perspectivas en 2024 y explorando la posibilidad de que supere a Estados Unidos como la economía más grande del mundo.

**Perspectivas Económicas de China en 2024:**

* **Contexto:** La economía china se encuentra en un punto de inflexión, marcada por la recuperación post-pandemia, la guerra en Ucrania, las tensiones geopolíticas y la desaceleración económica. 
* **Desafíos:** La economía enfrenta desafíos sin precedentes, incluyendo la caída del mercado inmobiliario, la disminución del consumo, la dificultad para pagar salarios en algunas áreas, la deuda pública creciente, la crisis de Evergrande, la baja tasa de natalidad y la desglobalización.
* **Oportunidades:** A pesar de los desafíos, se espera un crecimiento estable, un aumento de las exportaciones y la importancia de la tecnología como motor de crecimiento, con especial atención a la energía renovable, la digitalización y la innovación.
* **Divergencia de Opiniones:** Existe una divergencia de opiniones sobre el futuro económico de China. Expertos internacionales, especialmente en Europa, son más pesimistas, mientras que las voces dentro de China son más optimistas, con confianza en la capacidad de la economía para recuperarse y crecer.
* **El Papel del Gobierno:** El gobierno chino está implementando medidas para impulsar la economía, incluyendo políticas de estímulo para el sector inmobiliario, la apertura a la inversión extranjera y la emisión de deuda para apoyar a las regiones con dificultades.
* **Resiliencia:** A pesar de los desafíos, China ha demostrado una capacidad para superar las crisis y continuar su crecimiento económico.

**¿Algún día la economía de China alcanzará a la de Estados Unidos?**

* **Desafíos Internos:** La desaceleración económica, la deuda pública, la crisis inmobiliaria, la baja tasa de natalidad y el consumo interno débil son obstáculos importantes para el crecimiento de China.
* **Desafíos Externos:** La desglobalización, la disminución de la demanda global y la competencia estratégica con Estados Unidos crean un ambiente geopolítico complejo que afecta la economía china.
* **Posibles Escenarios:** Algunos expertos creen que China aún tiene potencial para superar a Estados Unidos, pero requiere una transformación económica hacia un modelo de crecimiento impulsado por el consumo y la innovación. Otros advierten que la desaceleración económica y los desafíos internos podrían impedir que China alcance a Estados Unidos, llevando a un estancamiento o incluso a una crisis.

**Conclusión:**

El futuro económico de China es incierto. Superar a Estados Unidos como la economía más grande del mundo es un objetivo ambicioso que enfrenta desafíos internos y externos. El éxito de China dependerá de su capacidad para reformar su modelo económico, controlar la deuda, estimular el consumo interno y adaptarse a un entorno global cambiante. La capacidad de China para superar las crisis y su papel crucial en la economía global sugieren que la historia de crecimiento del país aún no ha terminado. 

**Reflexiones:**

* La competencia económica entre Estados Unidos y China es un tema complejo con implicaciones globales.
* La economía china se encuentra en un punto de inflexión, donde deberá afrontar los desafíos de la desaceleración económica, la deuda pública y el cambio demográfico.
* La capacidad de China para adaptarse y transformarse será crucial para su futuro económico.


**Espacio de discusión: ¿Que conclusión obtenemos del ejercicio anterior?**

1. **Crecimiento Estable y Oportunidades de Exportación**: A pesar de los desafíos económicos, se espera un crecimiento estable en China para 2024, lo que puede favorecer las exportaciones de café debido a un aumento potencial en la demanda.

2. **Desafíos Económicos Internos y Externos**: La desaceleración económica, la deuda pública y las tensiones geopolíticas pueden afectar el consumo interno y la demanda de productos importados. Las empresas deben estar preparadas para posibles fluctuaciones en la demanda.

3. **Enfoque en Innovación y Digitalización**: La tecnología y la digitalización son cruciales para el crecimiento en China. Integrar e-commerce y marketing digital en las estrategias de exportación puede ser clave para captar consumidores chinos.

## **11. Juguemos con la temperatura** <a name="juguemos-con-la-temperatura"></a>

La temperatura es un parámetro que controla la "creatividad" o aleatoriedad del texto generado por un LLM. Una temperatura más alta (por ejemplo, 0.7) produce resultados más diversos y creativos, mientras que una temperatura más baja (por ejemplo, 0.2) hace que el resultado sea más determinista y enfocado. En la práctica, la temperatura afecta la distribución de probabilidad sobre los posibles tokens en cada paso del proceso de generación. Una temperatura de 0 haría que el modelo sea completamente determinista, eligiendo siempre el token más probable.

**Configuraciones de Temperatura y sus Usos**

- **Temperatura baja (0.1 - 0.4):**
  - Produce respuestas más enfocadas, conservadoras y consistentes.
  - Útil cuando se necesita información factual, respuestas precisas o mensajes que se adhieran estrictamente a un formato específico o a las directrices de una marca.

- **Temperatura moderada (0.5 - 0.7):**
  - Logra un equilibrio entre creatividad y consistencia.
  - Adecuada para la generación de contenido general, donde se desea una mezcla de precisión e inventiva.

- **Temperatura alta (0.8 - 1.0):**
  - Genera resultados más creativos, diversos e inesperados.
  - Preferida por los profesionales de marketing cuando se busca ideas innovadoras para campañas, contenido atractivo para redes sociales o nuevas perspectivas sobre un tema.

In [54]:
# Definir el prompt para la historia
prompt_historia = """
Completa la historia en máximo 250 carácteres: Una vez, en un bosque mágico, vivía un guerrero llamado Darth Vader. Un día, mientras exploraba, encontró un misterioso...
"""

In [55]:
# Obtener la continuación de la historia con temperatura baja
respuesta_gemini_baja = get_gemini_completion(prompt=prompt_historia, temperature=0.1)

# Obtener la continuación de la historia con temperatura media
respuesta_gemini_media = get_gemini_completion(prompt=prompt_historia, temperature=0.5)

# Obtener la continuación de la historia con temperatura alta
respuesta_gemini_alta = get_gemini_completion(prompt=prompt_historia, temperature=0.9)

In [56]:
# Mostrar el contenido como Markdown: Baja
display(Markdown(respuesta_gemini_baja))

...cristal que emitía un brillo azul. Al tocarlo, sintió una poderosa energía fluir por su cuerpo. Darth Vader se dio cuenta de que este cristal era la llave para... 


In [57]:
# Mostrar el contenido como Markdown: Media
display(Markdown(respuesta_gemini_media))

...cristal azul que emitía un brillo extraño. Al tocarlo, sintió un poder desconocido recorriendo su cuerpo. Darth Vader se dio cuenta de que este cristal podría ser la llave para dominar el universo. 


In [58]:
# Mostrar el contenido como Markdown: Alta
display(Markdown(respuesta_gemini_alta))

...cristal que emitía una luz roja. Al tocarlo, sintió un poder inmenso, pero también una oscuridad que lo atraía. Darth Vader se convirtió en un temible guerrero del lado oscuro. 


In [59]:
# Definir el prompt para la frase
prompt_frase = """
Responde en máximo 250 carácteres: Describe un mundo donde los árboles pueden hablar y las montañas pueden moverse.
"""

In [60]:
# Obtener la descripción con temperatura baja
respuesta_gemini_baja = get_gemini_completion(prompt=prompt_frase, temperature=0.1)

# Obtener la descripción con temperatura media
respuesta_gemini_media = get_gemini_completion(prompt=prompt_frase, temperature=0.5)

# Obtener la descripción con temperatura alta
respuesta_gemini_alta = get_gemini_completion(prompt=prompt_frase, temperature=0.9)

In [61]:
# Mostrar el contenido como Markdown: Baja
display(Markdown(respuesta_gemini_baja))

Un mundo donde los árboles susurran secretos al viento y las montañas se inclinan para escuchar. Los bosques son bibliotecas vivientes y las cumbres, guardianes silenciosos de la historia. La naturaleza es un ser consciente, lleno de sabiduría y misterio. 


In [62]:
# Mostrar el contenido como Markdown: Media
display(Markdown(respuesta_gemini_media))

Un mundo donde los árboles susurran secretos al viento y las montañas se inclinan para escuchar. La naturaleza se comunica, susurra historias antiguas y sus movimientos revelan misterios. La vida es un diálogo constante entre la tierra y sus habitantes. 


In [63]:
# Mostrar el contenido como Markdown: Alta
display(Markdown(respuesta_gemini_alta))

Un mundo donde los árboles susurran secretos al viento y las montañas se inclinan para escuchar, creando valles que se agitan con el latido del corazón de la tierra. Los ríos fluyen con la música de la naturaleza, y la luna danza con las estrellas en un cielo vivo y vibrante. 


## **12. Traducción de archivos Excel** <a name="traducción-de-archivos-excel"></a>

Una vez análizamos la económia China vamos a averiguar que requisitos deberiamos cumplir para exportar a china.
1. Vamos a https://globaltradehelpdesk.org/es.
2. Buscamos cómo exportar Derivados de café a China con la Partida 2101110010.
3. Vamos a Requisito de autorización por motivos sanitarios y fitosanitarios para importar determinados producto.
4. Miremos esta: Anuncio sobre la inspección de alimentos y aditivos alimentarios importados (2009, nº 72) 关于进口食品及食品添加剂检验的公告（2009年第72号.
5. Elijamos el primer resultado en google
6. Descarguemos los excels.
    Anexos:
    1. Catálogo de alimentos importados y normas aplicables correspondientes.xls
    2. Catálogo de aditivos alimentarios permitidos para su uso en mi país y normas aplicables correspondientes.xls
    3. Catálogo de especias alimentarias permitidas para su uso en mi país y normas aplicables correspondientes.xls

In [ ]:
df = pd.read_excel('doc5515.xls',skiprows=2)

In [ ]:
df.head()

In [ ]:
def traducir_dataframe_json(df):
    # Convertir el DataFrame a JSON, incluyendo NaNs y números
    df_json = df.to_json(orient="split", index=False)
    
    # Formar el prompt para la API, indicando que se espera un JSON en la respuesta
    prompt = f"Traduce el siguiente DataFrame de JSON del chino simplificado al español y devuelve el resultado en el mismo formato JSON:\n\n{df_json}"
    
    # Realizar la llamada a la API
    response = get_gemini_completion(prompt=prompt, temperature=0.1)
    return response

In [ ]:
df_10_filas = df.head(10)

In [ ]:
# Traducir el DataFrame
texto_traducido = traducir_dataframe_json(df)
if texto_traducido:
    try:
        # Extraer JSON del texto traducido
        start = texto_traducido.find('{')
        end = texto_traducido.rfind('}') + 1
        json_str = texto_traducido[start:end]
        
        # Cargar el JSON a un diccionario de Python
        data = json.loads(json_str)
        
        # Convertir el diccionario a un DataFrame
        df_traducido_gemini = pd.read_json(json.dumps(data), orient="split")
        
        # Guardar el DataFrame traducido si es necesario
        df_traducido_gemini.to_excel("archivo_traducido_gemini.xlsx", index=False)
        
    except (json.JSONDecodeError, ValueError) as e:
        print(f"Error al procesar la respuesta JSON: {str(e)}")
        df_traducido = None

In [ ]:
# import pandas as pd
# import json
# from concurrent.futures import ThreadPoolExecutor

# def traducir_parte(df_chunk):
#     # Imprimir las filas que se están traduciendo
#     print(f"Traduciendo las filas: \n{df_chunk}")
    
#     df_json = df_chunk.to_json(orient="split", index=False)
#     prompt = f"Traduce el siguiente DataFrame de JSON del chino simplificado al español y devuelve el resultado en el mismo formato JSON:\n\n{df_json}"
#     response = get_gemini_completion(prompt=prompt)
    
#     if response:
#         try:
#             start = response.find('{')
#             end = response.rfind('}') + 1
#             json_str = response[start:end]
#             data = json.loads(json_str)
#             df_traducido_chunk = pd.read_json(json.dumps(data), orient="split")
#             return df_traducido_chunk
#         except (json.JSONDecodeError, ValueError) as e:
#             print(f"Error al procesar la respuesta JSON: {str(e)}")
#             return pd.DataFrame()
#     return pd.DataFrame()

# def traducir_dataframe_json(df, chunk_size=100):
#     df_traducido_total = pd.DataFrame()
#     with ThreadPoolExecutor(max_workers=4) as executor:
#         futures = []
#         for start_row in range(0, len(df), chunk_size):
#             print(f"Procesando filas de {start_row} a {min(start_row + chunk_size, len(df))}")
#             end_row = min(start_row + chunk_size, len(df))
#             df_chunk = df.iloc[start_row:end_row]
#             futures.append(executor.submit(traducir_parte, df_chunk))
        
#         for future in futures:
#             df_traducido_chunk = future.result()
#             df_traducido_total = pd.concat([df_traducido_total, df_traducido_chunk], ignore_index=True)
    
#     return df_traducido_total

# df_traducido = traducir_dataframe_json(df)

# # Guardar el DataFrame traducido si es necesario
# if not df_traducido.empty:
#     df_traducido.to_excel("archivo_traducido.xlsx", index=False)